# 1D CNN for 12-Lead ECG Classification

**Main Thesis Topic:** “Zero-shot classification of ECG signals using CLIP-like model”. 

**For example:** Train on PBT-XL: 

- **Text Encoder:** ClinicalBERT
- **Image Encoder:** 1D CNN

**Experiment A):** Baseline: We can take only the name of the class. For example, take “Myocardial Infarction” as a text. We should exclude some classes from training and after training is completed, the CLIP-like model can be tested on these excluded classes. 
Next, we get embeddings of text from biobert and train ECG encoder with contrastive loss.

**Experiment B):** Same as Experiment A but instead of testing on the same dataset/classes, we would test on other datasets containing different classes.

**Evaluation metrics:** 
- Main: AUC-ROC, average_precison_score, 
- Optional: Specificity, Sensitivity, F1-score 

Outcome: 

It’s possible to train CLIP-like models with freezed (or unchanged/not fine tuned for downstream tasks) text encoder
Training ECG encoders that are viable for representing different domains (within ECG modality) and previously unseen classes. 
Training a CLIP-like model on ECGs has little novelty.

https://www.baeldung.com/cs/contrastive-learning

https://scikit-learn.org/stable/modules/generated/sklearn.metrics.average_precision_score.html



In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import sys

c:\Users\navme\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\navme\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Users\navme\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
sys.path.append('C:/Users/navme/Desktop/ECG_Project/PyFiles')

In [9]:
# Import helper functions
import helper_functions as hf
import helper_function_tests as hft
from dataset import PhysioNetDataset

In [10]:
PhysioNet_PATH = f'C:/Users/navme/Desktop/ECG_Thesis_Local/PhysioNet-2021-Challenge/physionet.org/files/challenge-2021/1.0.3/training'
PhysioNet_PATH

'C:/Users/navme/Desktop/ECG_Thesis_Local/PhysioNet-2021-Challenge/physionet.org/files/challenge-2021/1.0.3/training'

In [15]:
train_set = PhysioNetDataset(PhysioNet_PATH, train = True)
test_set = PhysioNetDataset(PhysioNet_PATH, train = False)

print("There are {0} datasets in the train_set and {1} datasets in test_set".format(len(train_set), len(test_set)))

There are 7 datasets in the train_set and 2 datasets in test_set


## Model